Hey Esther! Here is the code for the Random Forest model that we worked on on Wednesday

**The line of code:** 

<div class="alert alert-block alert-info">
    <b># Resample the training data with the BalancedRandomForestClassifier</b>
<br>
from imblearn.ensemble import BalancedRandomForestClassifier
<br>
brfc = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
<br>
rf = brfc.fit(X_train,y_train)
</div>

**takes so long to run, just a warning!**

## Import dependencies and read csv

I saved the df that nisha and suweatha queried into a csv and imported it into this notebook just so our analysis can be a little cleaner

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path='stack_overflow.csv'
df=pd.read_csv(file_path)

In [3]:
df.head()

,Unnamed: 0,q_id,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id,q_day,q_hour,q_hour_min,accepted_answer_duration
0,0,67341742,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,Saturday,0,00:03,0.192894
1,1,67341817,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,67341857,Saturday,0,00:17,0.149909
2,2,67341895,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,Saturday,0,00:32,0.076868
3,3,67341936,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,67341961,Saturday,0,00:41,0.088950
4,4,67341921,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,67341974,Saturday,0,00:38,0.166569


## Data Preprocessing

- Dropped null values and dropped uncessary columns
- Binned data in accepted_answer_duration

In [4]:
df=df.dropna()

In [5]:
rf_df=df.drop(['q_id','accepted_answer_id'], axis=1)
rf_df.head()

,Unnamed: 0,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration
0,0,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,Saturday,0,00:03,0.192894
1,1,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,Saturday,0,00:17,0.149909
2,2,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,Saturday,0,00:32,0.076868
3,3,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,Saturday,0,00:41,0.088950
4,4,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,Saturday,0,00:38,0.166569


In [37]:
stats_df=rf_df[['q_score_tier','accepted_answer_duration']]
stats_df.head()

,q_score_tier,accepted_answer_duration
0,Zero Score (0),0.192894
1,Zero Score (0),0.149909
2,Positive Score (>0),0.076868
3,Positive Score (>0),0.088950
4,Positive Score (>0),0.166569


In [50]:
rf_df.sort_values(by='accepted_answer_duration',ascending=False)

,Unnamed: 0,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration
207028,207957,Zero Score (0),Medium (50-100),Medium (10-20),30-40,98,50-100,2,Thursday,14,14:36,2856.969993
201353,202268,Zero Score (0),Medium (50-100),Short (0 - 10),50-16000,140,100-250,4,Monday,8,08:18,2856.162988
209338,210268,Positive Score (>0),Long (100-150),Medium (10-20),50-16000,140,100-250,4,Sunday,4,04:49,2848.648400
195545,196437,Zero Score (0),Medium (50-100),Short (0 - 10),50-16000,228,100-250,5,Sunday,10,10:37,2774.281540
198004,198903,Positive Score (>0),Short (0 - 50),Short (0 - 10),50-16000,818,500-10000,5,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...,...,...,...,...,...,...
118903,119495,Zero Score (0),Medium (50-100),Short (0 - 10),10-20,95,50-100,3,Thursday,23,23:38,0.004629
204995,205921,Zero Score (0),Short (0 - 50),Short (0 - 10),20-30,69,50-100,3,Wednesday,11,11:27,0.003704
171188,171983,Positive Score (>0),Medium (50-100),Short (0 - 10),30-40,149,100-250,2,Tuesday,16,16:30,0.000194
165801,166581,Positive Score (>0),Medium (50-100),Medium (10-20),10-20,41,<50,2,Friday,19,19:34,0.000191


In [52]:
answer_bins = [0, 1, 24, 3000]
answer_bins_group_names = ["<1H", "<1D", ">1D"]

# Categorize score based on the bins.
rf_df['accepted_answer_duration_bin'] = pd.cut(rf_df['accepted_answer_duration'], answer_bins, labels=answer_bins_group_names)

In [53]:
rf_df.head()

,Unnamed: 0,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration,accepted_answer_duration_bin
0,0,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,Saturday,0,00:03,0.192894,<1H
1,1,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,Saturday,0,00:17,0.149909,<1H
2,2,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,Saturday,0,00:32,0.076868,<1H
3,3,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,Saturday,0,00:41,0.088950,<1H
4,4,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,Saturday,0,00:38,0.166569,<1H


In [39]:
stats_df.groupby('q_score_tier').mean()

,accepted_answer_duration
q_score_tier,
Negative Score (<0),12.539322
Positive Score (>0),27.312780
Zero Score (0),26.289773


In [55]:
rf_df.groupby('accepted_answer_duration_bin').count()

,Unnamed: 0,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,q_day,q_hour,q_hour_min,accepted_answer_duration
accepted_answer_duration_bin,,,,,,,,,,,,
<1H,111305,111305,111305,111305,111305,111305,111305,111305,111305,111305,111305,111305
<1D,71404,71404,71404,71404,71404,71404,71404,71404,71404,71404,71404,71404
>1D,26711,26711,26711,26711,26711,26711,26711,26711,26711,26711,26711,26711


## Create features and encode our features using pd.get_dummies

In [65]:
# Create our features
X = rf_df.drop(['accepted_answer_duration','accepted_answer_duration_bin','Unnamed: 0'], axis=1)
X = pd.get_dummies(X)

# Create our target
y = rf_df["accepted_answer_duration_bin"]

X.head()

,q_body_word_count,q_tags_count,q_hour,q_score_tier_Negative Score (<0),q_score_tier_Positive Score (>0),q_score_tier_Zero Score (0),q_title_char_count_bin_Long (100-150),q_title_char_count_bin_Medium (50-100),q_title_char_count_bin_Short (0 - 50),q_title_word_count_bin_Long (20-30),...,q_hour_min_23:50,q_hour_min_23:51,q_hour_min_23:52,q_hour_min_23:53,q_hour_min_23:54,q_hour_min_23:55,q_hour_min_23:56,q_hour_min_23:57,q_hour_min_23:58,q_hour_min_23:59
0,217,4,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47,4,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,423,2,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,243,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,57,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
# Check the balance of our target values
y.value_counts()

<1H    111305
<1D     71404
>1D     26711
Name: accepted_answer_duration_bin, dtype: int64

## Split data to training and testing sets

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

## Fit model: Random Forest Classifier

In [68]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brfc = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
rf = brfc.fit(X_train,y_train)

## Calculate Accuracy

In [69]:
# Calculated the balanced accuracy score
y_pred=rf.predict(X_test)
ba_balanced_forest=balanced_accuracy_score(y_test,y_pred)
ba_balanced_forest

0.42728989947746143

Accuracy of the model using y=accepted_answer_duration is **43%**

## Display Confusion Matrix


### Confusion Matrix with 3 Categories

(this is a little confusing, we should either bin it into 2 categories for ease of analysis or use the expected answer duration as our y value and bin that into 2 categories)

|                           | Predicted Negative Score (<0)  |Predicted Zero Score (0)|Predicted Positive Score (>0)|
| --------------------------| ------------------------------ |------------------------|-----------------------------|
| Actual Negative Score (<0)|    TRUE POSITIVE               |    FALSE NEGATIVE      | FALSE NEGATIVE              |
| Actual Zero Score (0)     |                                |     TRUE POSITIVE      | TRUE POSITIVE
|Actual Positive Score (>0)	|

In [63]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

cm_df=pd.DataFrame(cm,
                  index=["Actual <1H", "Actual <1D', "Actual >1D"],
                  columns=["Predicted <1H", "Predicted <1D","Predicted >1D"])
cm_df

,Predicted Negative Score (<0),Predicted Zero Score (0),Predicted Positive Score (>0)
Actual Negative Score (<0),4914,6870,6067
Actual Zero Score (0),6230,15038,6558
Actual Positive Score (>0),1356,1888,3434


## Print additional scores for analysis: precision, recall, and f1

In [13]:
# Print the imbalanced classification report
icr_balanced_forest=classification_report_imbalanced(y_test,y_pred)
print(icr_balanced_forest)

                           pre       rec       spe        f1       geo       iba       sup

Negative Score (<0)       0.15      0.46      0.70      0.23      0.57      0.32      5290
Positive Score (>0)       0.43      0.42      0.70      0.42      0.54      0.29     18059
     Zero Score (0)       0.65      0.40      0.73      0.50      0.54      0.28     29006

        avg / total       0.52      0.42      0.72      0.44      0.55      0.29     52355



In [14]:
#Summary of findings

print(f'For the Balanced Random Forest Classifier algortihm, the balanced accuracy score is {ba_balanced_forest}' 
      f'\n\nand the imbalanced classifcation report is:\n\n{icr_balanced_forest}')

For the Balanced Random Forest Classifier algortihm, the balanced accuracy score is 0.4294338073654384

and the imbalanced classifcation report is:

                           pre       rec       spe        f1       geo       iba       sup

Negative Score (<0)       0.15      0.46      0.70      0.23      0.57      0.32      5290
Positive Score (>0)       0.43      0.42      0.70      0.42      0.54      0.29     18059
     Zero Score (0)       0.65      0.40      0.73      0.50      0.54      0.28     29006

        avg / total       0.52      0.42      0.72      0.44      0.55      0.29     52355



## Feature Importance

In [70]:
# List the features sorted in descending order by feature importance
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.1143007447923381, 'q_body_word_count'),
 (0.08267488028286278, 'q_hour'),
 (0.04466696447684592, 'q_tags_count'),
 (0.015818130000895055, 'q_view_count_bin_50-16000'),
 (0.010423910203227636, 'q_day_Wednesday'),
 (0.01027358000781115, 'q_day_Thursday'),
 (0.009970748086731922, 'q_day_Tuesday'),
 (0.00978441609260875, 'q_day_Monday'),
 (0.008492565597447658, 'q_title_word_count_bin_Medium (10-20)'),
 (0.008455112389517162, 'q_day_Saturday'),
 (0.00820670359906295, 'q_title_word_count_bin_Short (0 - 10)'),
 (0.00790699978126666, 'q_score_tier_Zero Score (0)'),
 (0.007754533195261118, 'q_view_count_bin_20-30'),
 (0.007751584678058564, 'q_day_Friday'),
 (0.007732850090337837, 'q_score_tier_Positive Score (>0)'),
 (0.007723483563316564, 'q_day_Sunday'),
 (0.0069672675556941895, 'q_view_count_bin_30-40'),
 (0.0068496582127536695, 'q_title_char_count_bin_Medium (50-100)'),
 (0.005872625194443157, 'q_title_char_count_bin_Short (0 - 50)'),
 (0.005781008670643229, 'q_body_len_bin_100-250'),
